# FEATURING

## I.Các bước thực hiện

1. Load dữ liệu COVID-19 đã clean

2. Tạo các đặc trưng (features) phục vụ phân tích & dự báo

3. Kiểm tra dữ liệu sau mỗi bước

4. Lưu dataset đã feature engineering

## II.Import thư viện & load dữ liệu

In [5]:
import pandas as pd
import os

# Input / Output paths (fix relative path to project data folder)
PROCESSED_DATA_PATH = "../data/2_processed"
MODEL_DATA_PATH = "../data/3_model"

## III.Load dữ liệu đã clean

In [6]:
df = pd.read_csv(
    os.path.join(PROCESSED_DATA_PATH, "04_covid19_cleaned_long.csv"),
    parse_dates=["Date"]
)
df.head()

,Country/Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,2020-01-22,0,0,0
1,Albania,2020-01-22,0,0,0
2,Algeria,2020-01-22,0,0,0
3,Andorra,2020-01-22,0,0,0
4,Angola,2020-01-22,0,0,0


## IV.Sort dữ liệu theo Country + Date

In [7]:
df = df.sort_values(
    by=["Country/Region", "Date"]
).reset_index(drop=True)
df.groupby("Country/Region").head(3)

,Country/Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,2020-01-22,0,0,0
1,Afghanistan,2020-01-23,0,0,0
2,Afghanistan,2020-01-24,0,0,0
1143,Albania,2020-01-22,0,0,0
1144,Albania,2020-01-23,0,0,0
...,...,...,...,...,...
227458,Zambia,2020-01-23,0,0,0
227459,Zambia,2020-01-24,0,0,0
228600,Zimbabwe,2020-01-22,0,0,0
228601,Zimbabwe,2020-01-23,0,0,0


## V.Feature Engineering

### Ca nhiễm mới mỗi ngày

In [9]:
df["New_Cases"] = (
    df.groupby("Country/Region")["Confirmed"]
      .diff()
      .fillna(0)
)
df.groupby("New_Cases").head()

,Country/Region,Date,Confirmed,Deaths,Recovered,New_Cases
0,Afghanistan,2020-01-22,0,0,0,0.0
1,Afghanistan,2020-01-23,0,0,0,0.0
2,Afghanistan,2020-01-24,0,0,0,0.0
3,Afghanistan,2020-01-25,0,0,0,0.0
4,Afghanistan,2020-01-26,0,0,0,0.0
...,...,...,...,...,...,...
229294,Zimbabwe,2021-12-16,182057,4759,0,4367.0
229295,Zimbabwe,2021-12-17,189567,4779,0,7510.0
229298,Zimbabwe,2021-12-20,195079,4805,0,3406.0
229299,Zimbabwe,2021-12-21,197768,4819,0,2689.0


### Ca tử vong mới mỗi ngày

In [10]:
df["New_Deaths"] = (
    df.groupby("Country/Region")["Deaths"]
      .diff()
      .fillna(0)
)
df.groupby("New_Deaths").head()

,Country/Region,Date,Confirmed,Deaths,Recovered,New_Cases,New_Deaths
0,Afghanistan,2020-01-22,0,0,0,0.0,0.0
1,Afghanistan,2020-01-23,0,0,0,0.0,0.0
2,Afghanistan,2020-01-24,0,0,0,0.0,0.0
3,Afghanistan,2020-01-25,0,0,0,0.0,0.0
4,Afghanistan,2020-01-26,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...
217559,United Kingdom,2021-02-14,4049910,142709,10918,10994.0,607.0
217560,United Kingdom,2021-02-15,4059691,143327,10963,9781.0,618.0
223444,Vietnam,2021-08-03,174461,2071,50831,16954.0,765.0
223463,Vietnam,2021-08-22,348059,8277,0,11352.0,737.0


### Moving Average 7 ngày (New Cases)

In [13]:
df["MA7_New_Cases"] = (
    df.groupby("Country/Region")["New_Cases"]
      .rolling(window=7)
      .mean()
      .reset_index(level=0, drop=True)
)
df["MA7_New_Cases"] = df["MA7_New_Cases"].fillna(0)
df.groupby("MA7_New_Cases").head()

,Country/Region,Date,Confirmed,Deaths,Recovered,New_Cases,New_Deaths,MA7_New_Cases,Mortality_Rate
0,Afghanistan,2020-01-22,0,0,0,0.0,0.0,0.000000,0.000000
1,Afghanistan,2020-01-23,0,0,0,0.0,0.0,0.000000,0.000000
2,Afghanistan,2020-01-24,0,0,0,0.0,0.0,0.000000,0.000000
3,Afghanistan,2020-01-25,0,0,0,0.0,0.0,0.000000,0.000000
4,Afghanistan,2020-01-26,0,0,0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
229332,Zimbabwe,2022-01-23,228254,5294,0,75.0,2.0,310.857143,0.023193
229368,Zimbabwe,2022-02-28,236380,5395,0,577.0,2.0,401.285714,0.022823
229372,Zimbabwe,2022-03-04,238739,5397,0,1236.0,1.0,467.428571,0.022606
229378,Zimbabwe,2022-03-10,241548,5408,0,1205.0,8.0,577.857143,0.022389


### Tỉ lệ tử vong

In [14]:
df["Mortality_Rate"] = (
    df["Deaths"] /
    df["Confirmed"].replace(0, pd.NA)
)
df["Mortality_Rate"] = df["Mortality_Rate"].fillna(0)
df.groupby("Mortality_Rate").head()

C:\Users\Bao\AppData\Local\Temp\ipykernel_19040\670051252.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Mortality_Rate"] = df["Mortality_Rate"].fillna(0)


,Country/Region,Date,Confirmed,Deaths,Recovered,New_Cases,New_Deaths,MA7_New_Cases,Mortality_Rate
0,Afghanistan,2020-01-22,0,0,0,0.0,0.0,0.000000,0.000000
1,Afghanistan,2020-01-23,0,0,0,0.0,0.0,0.000000,0.000000
2,Afghanistan,2020-01-24,0,0,0,0.0,0.0,0.000000,0.000000
3,Afghanistan,2020-01-25,0,0,0,0.0,0.0,0.000000,0.000000
4,Afghanistan,2020-01-26,0,0,0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
229736,Zimbabwe,2023-03-03,264127,5668,0,0.0,0.0,29.428571,0.021459
229737,Zimbabwe,2023-03-04,264127,5668,0,0.0,0.0,29.428571,0.021459
229738,Zimbabwe,2023-03-05,264127,5668,0,0.0,0.0,29.428571,0.021459
229741,Zimbabwe,2023-03-08,264276,5671,0,149.0,3.0,21.285714,0.021459


## VI.Kiểm tra dữ liệu sau feature engineering

In [15]:
df.isna().sum()
df.head()

,Country/Region,Date,Confirmed,Deaths,Recovered,New_Cases,New_Deaths,MA7_New_Cases,Mortality_Rate
0,Afghanistan,2020-01-22,0,0,0,0.0,0.0,0.0,0.0
1,Afghanistan,2020-01-23,0,0,0,0.0,0.0,0.0,0.0
2,Afghanistan,2020-01-24,0,0,0,0.0,0.0,0.0,0.0
3,Afghanistan,2020-01-25,0,0,0,0.0,0.0,0.0,0.0
4,Afghanistan,2020-01-26,0,0,0,0.0,0.0,0.0,0.0


## VII.Lưu dữ liệu

In [16]:
output_path = os.path.join(
    MODEL_DATA_PATH,
    "covid19_feature_engineered.csv"
)

df.to_csv(output_path, index=False)
print("✅ Feature engineering completed!")
print("📁 Saved to:", output_path)

✅ Feature engineering completed!
📁 Saved to: ../data/3_model\covid19_feature_engineered.csv
